In [1]:
#Mise en place de la dataframe Pandas (exécution environ 10s)

import pandas as pd
import numpy as np
import geopandas as gpd
import matplotlib.pyplot as plt
import requests
import io

json_url = "https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/exports/json?lang=fr&timezone=Europe%2FParis&select=equip_nom%2C%20equip_service_date%2C%20equip_douche%2C%20equip_sanit%2C%20equip_travaux_date&where=reg_nom%20like%20%27%C3%8Ele-de-France%27"
req = requests.get(json_url)
json_data = req.json()
df_dirty = pd.DataFrame(json_data)
df_dirty['equip_service_date'] = pd.to_numeric(df_dirty['equip_service_date'], errors='coerce') # Conversion de variable en float
df_dirty['equip_travaux_date'] = pd.to_numeric(df_dirty['equip_travaux_date'], errors='coerce') # Conversion de variable en float
dff = df_dirty.copy()
dff['equip_service_date'] = dff['equip_service_date'].fillna(0) # Remplacement des NaN par 0
dff['equip_travaux_date'] = dff['equip_travaux_date'].fillna(0) # Remplacement des NaN par 0

In [ ]:
#Test
df = dff[dff['equip_service_date'] != 0]
df = df.copy()
df.count()

equip_nom             23218
equip_service_date    23218
equip_douche          22444
equip_sanit           23079
equip_travaux_date    23218
dtype: int64

In [ ]:
# Conversion en bool des variables de qualité
df['equip_douche'] = (df['equip_douche'].astype(bool)).astype(int)
df['equip_sanit'] = (df['equip_sanit'].astype(bool)).astype(int)

In [ ]:
# Variable représentant la dernière rénovation en date (equip_travaux_date vaut 0 s'il n'y a jamais eu de travaux)
df['derniere_renov'] = np.maximum(df['equip_travaux_date'], df['equip_service_date'])

In [ ]:
# Min et max de la dernière rénovation
borne_min = df['derniere_renov'].min()
borne_max = df['derniere_renov'].max()

In [ ]:
# Création des var. d'ancienneté
df['coef_recent'] = (df['derniere_renov'] - borne_min)/(borne_max - borne_min) # Coef. représentant l'ancienneté d'une infrastructure en comparaison aux autres de l'échantillon
df['récent'] = (df['derniere_renov']>=2011).astype(int)
df['moyen'] = ((df['derniere_renov']>=1986) & (df['derniere_renov']<2011)).astype(int)
df['ancien'] = ((df['derniere_renov'] >= borne_min) & (df['derniere_renov'] < 1986)).astype(int) 

In [7]:
#Test
df.iloc[1000]

equip_nom             BASSIN D'INITIATION DE 12,50 METRES
equip_service_date                                 1999.0
equip_douche                                            0
equip_sanit                                             1
equip_travaux_date                                    0.0
derniere_renov                                     1999.0
coef_recent                                      0.927171
récent                                                  0
moyen                                                   1
ancien                                                  0
Name: 1286, dtype: object

In [ ]:
# Définition du coeff de vétusté
df['qualite'] = 0.8 * df['récent'] + 0.4 * df['moyen'] + 0.1 * df['equip_douche'] + 0.1 * df['equip_sanit']

In [ ]:
#Test
df.head(10)

,equip_nom,equip_service_date,equip_douche,equip_sanit,equip_travaux_date,derniere_renov,coef_recent,récent,moyen,ancien,qualite
1,TERRAIN DE TENNIS N°1,1976.0,1,1,0.0,1976.0,0.862745,0,0,1,0.2
2,COURTS DE TENNIS 8-9-10 2,1950.0,1,1,0.0,1950.0,0.789916,0,0,1,0.2
4,CITY STADE,2009.0,1,1,0.0,2009.0,0.955182,0,1,0,0.6
5,ESPACE COACHING,2013.0,1,1,0.0,2013.0,0.966387,1,0,0,1.0
6,Salle de renforcement musculaire,2023.0,1,1,0.0,2023.0,0.994398,1,0,0,1.0
7,Salle de biking,2023.0,1,1,0.0,2023.0,0.994398,1,0,0,1.0
8,SALLE RPM IMMERSIVE,1989.0,1,1,2014.0,2014.0,0.969188,1,0,0,1.0
9,1 COURT DE TENNIS,1992.0,1,1,0.0,1992.0,0.907563,0,1,0,0.6
10,SALLE DE DANSE,1996.0,1,1,2010.0,2010.0,0.957983,0,1,0,0.6
11,SALLE DE MUSCULATION CLUB,1994.0,1,1,0.0,1994.0,0.913165,0,1,0,0.6


In [10]:
# Calcul moyenne et médiane d'ancienneté
moyenne = df['coef_recent'].mean()
mediane = df['coef_recent'].median()
[moyenne, mediane]
# On voit qu'en IDF, les infrastructures sont très récentes :)

[np.float64(0.9169621849945941), np.float64(0.9299719887955182)]

In [11]:
# Ne conserver que les anciennetés inférieures à un certain seuil
df_filtre = df[df['coef_recent'] < 0.7]
df_filtre.head(20)

,equip_nom,equip_service_date,equip_douche,equip_sanit,equip_travaux_date,derniere_renov,coef_recent,récent,moyen,ancien,qualite
120,TERRAIN DE BASKET-BALL,1897.0,1,1,0.0,1897.0,0.641457,0,0,1,0.2
197,TERRAIN DE BOULES,1909.0,1,1,0.0,1909.0,0.675070,0,0,1,0.2
217,TERRAIN DE RUGBY SYNTHETIQUE,1902.0,1,1,0.0,1902.0,0.655462,0,0,1,0.2
1133,MANEGE,1906.0,1,1,0.0,1906.0,0.666667,0,0,1,0.2
1184,GYMNASE,1894.0,1,1,0.0,1894.0,0.633053,0,0,1,0.2
1185,TERRAIN DE MULTISPORTS 1,1894.0,1,1,0.0,1894.0,0.633053,0,0,1,0.2
1186,TERRAIN DE HANDBALL 2,1884.0,1,1,0.0,1884.0,0.605042,0,0,1,0.2
1187,TERRAIN DE HANDBALL 1,1884.0,1,1,0.0,1884.0,0.605042,0,0,1,0.2
1188,ZONE DE SAUT EN LONGUEUR 2,1884.0,1,1,0.0,1884.0,0.605042,0,0,1,0.2
1195,GYMNASE,1879.0,1,1,0.0,1879.0,0.591036,0,0,1,0.2


In [12]:
# equip_nom, equip_service_date, equip_douche, equip_sanit, equip_travaux_date

In [13]:
# Lien json
# https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/exports/json?lang=fr&timezone=Europe%2FParis&select=equip_nom%2C%20equip_service_date%2C%20equip_douche%2C%20equip_sanit%2C%20equip_travaux_date&where=reg_nom%20like%20%27%C3%8Ele-de-France%27

# Lien CSV
# https://equipements.sports.gouv.fr/api/explore/v2.1/catalog/datasets/data-es/exports/csv?lang=fr&timezone=Europe%2FParis&use_labels=true&delimiter=%3B&select=equip_nom%2C%20equip_service_date%2C%20equip_douche%2C%20equip_sanit%2C%20equip_travaux_date&where=reg_nom%20like%20%27%C3%8Ele-de-France%27